In [2]:
import ee
import geemap

In [5]:
Map = geemap.Map()

In [6]:
ee.Initialize()

In [7]:
def get_square_roi(lat, lon):
    # Convert the lat-long point to an EE geometry object
    point = ee.Geometry.Point(lon, lat)
    
    # Define the size of the ROI in meters
    roi_size = 1280
    
    # Create a square buffer around the point with the given size
    roi = point.buffer(roi_size/2).bounds().getInfo()['coordinates']
    
    # Define the visualization parameters for the map
    vis_params = {'color': 'red'}
    
    # Create a map centered on the point with the square ROI overlaid
    roi_map = geemap.Map(center=[lat, lon], zoom=15)
    roi_map.addLayer(point, {}, 'Center point')
    roi_map.addLayer(ee.Geometry.Polygon(roi), vis_params, 'ROI')
    
    # Return the square ROI as a list of coordinates
    return roi

In [8]:
roi = get_square_roi(32, 53)
roi = ee.Geometry.Polygon(roi, None, False)
Map = geemap.Map(zoom=12)
Map.addLayer(roi)
Map.centerObject(roi)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterDate('2021-07-15', '2021-08-15') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE',10)) \
                .filterBounds(roi) \
                .filter(ee.Filter.contains('.geo', roi)) #this line checks if the scene completly covers the roi, which mean roi is in the scene

In [10]:
s2_img = s2.median().clip(roi).multiply(0.0001)
bands = ['B1','B2','B3','B4']
s2_img = s2_img.select(bands)

In [11]:
geemap.download_ee_image(s2_img, "s2.tif",crs='EPSG:3857', scale=10,region = roi)

s2.tif: |          | 0.00/734k (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [12]:
from skimage import io
s2_np = io.imread('s2.tif')
print(s2_np.shape)

(152, 151, 4)


In [ ]:
def download_sentinel2_image(lat, lon):
    # Initialize Earth Engine
    ee.Initialize()

    # Create a point geometry for the given lat/lon coordinates
    point = ee.Geometry.Point(lon, lat)

    # Load Sentinel-2 imagery for the given point and filter by date
    s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(point)\
        .filterDate('2019-01-01', '2021-12-31')\
        .sort('CLOUD_COVERAGE_ASSESSMENT')\
        .first()

    # Select the red, green, and blue bands
    s2_rgb = s2.select('B4', 'B3', 'B2')

    # Get the image URL and download the image as a TIFF file
    url = s2_rgb.getDownloadUrl({
        'scale': 10,
        'region': point.buffer(64).bounds().getInfo()['coordinates'],
        'filePerBand': False,
        'format': 'tiff'
    })

    geemap.download_from_url(url, file_path='./sentinel2_image.tif')
    print('Sentinel-2 image downloaded successfully!')

In [ ]:
download_sentinel2_image(32, 53)